In [1]:
import numpy as np 
import pandas as pd 
import mlflow
import mlflow.xgboost
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

import optuna
from optuna.integration.mlflow import MLflowCallback
import joblib

from typing import Optional, Dict, Tuple, Literal
from enefit_challenge.models.forecaster import Forecaster
from enefit_challenge.utils.dataset import load_enefit_training_data

import warnings
warnings.filterwarnings('ignore')


TRACKING_URI = "http://127.0.0.1:5000/" # local tracking URI -> launch mlflow before training 

/Users/dylantartarini/miniconda3/envs/enefit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## XGBoost and Categorical Variables

From previous experiments, we noticed that `XGBoostForecaster` produces results comparable to `CatBoostForecaster`, and does so without the usage of categorical variables, which are not allowed automatically in the `XGBRegressor` base model.  

Goal of this notebook is to expand the `XGBoostForecaster` class to enable it to use categorical features and therefore put it on equal ground with the other baseline class `CatBoostForecaster`.  

Our guess is, with access to categorical features, the XGBoost version will perform even better than CatBoost.

### How to treat Categorical Variables?
Categorical variables are `['county', 'product_type']`; since they do not have a lot of cardinality, there are at least three obviuos way to treat them:
1. using the `enable_categorical=True` parameter in the XGBRegressor, after having transformed columns into categorical columns in training dataframe using `df[col].astype('category')`;
2. using OneHotEconding (`pd.get_dummies()`)
3. keeping them as they are and casting values to `int`

## Method 1: Categorical Enabling

This method does not seem to work with the XGBRegressor -> need to test the other one 

In [2]:
df_train = load_enefit_training_data()

not_feature_columns = ['datetime', 'row_id','prediction_unit_id','date','time', 'data_block_id']
cat_columns = ['county', 'product_type']

In [3]:
# Convert categorical columns to category type
for col in cat_columns:
    df_train[col] = df_train[col].astype('category')

In [8]:
df_train["county"].dtype

CategoricalDtype(categories=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], ordered=False, categories_dtype=int64)

In [6]:
df_train["county"].value_counts()

county
0     212872
11    197948
7     173042
5     151592
15    148714
4     147226
10    134604
14    125776
3     122464
9     122464
13    121024
2     115170
1      91848
8      91848
6      30616
12     30616
Name: count, dtype: int64

In [9]:
df_train["product_type"].dtype

CategoricalDtype(categories=[0, 1, 2, 3], ordered=False, categories_dtype=int64)

In [10]:
df_train["product_type"].value_counts()

product_type
3    918480
1    781428
0    170500
2    147416
Name: count, dtype: int64

In [21]:
class XGBoostForecaster(Forecaster):
    """
        Implementaiton of a Forecaster using `XGBRegressor` as base model, 
        `optuna` for hyperparameters optimization and `mlflow` as backend to track experiments
        and register best-in-class model for time series prediction.
    """
    def __init__(self)-> None:
        self.tracking_uri = mlflow.set_tracking_uri(TRACKING_URI)
        pass

    def fit_model(
        self,  
        X:pd.DataFrame,
        y:pd.Series,
        params:Optional[Dict]=None,
    ) -> XGBRegressor:
        """
        Trains a `XGBRegressor`

        -------     
        params:
        -------
        `X`:`pd.DataFrame`
            Features to use for fitting
        `y`:`pd.Series`
            Target variable
        `params`: `Optional[Dict]`
            optional dictionary of parameters to use
        -------     
        returns:
        -------
        fitted `XGBRegressor`
        """
        model = XGBRegressor(
            n_estimators=100, 
            objective='reg:squarederror',
            enable_categorical=True,
            tree_method="approx"
        )
        if params:
            model.set_params(**params)

        model.fit(X, y)
    
        return model
    
    def fit_and_test_fold(
        self, 
        params:Dict,
        X: pd.DataFrame, 
        y: pd.Series, 
        year_month_train, 
        year_month_test,
        categorical_features: list=[],
        experiment_name: str="xgboost",
        artifact_path: str="xgboost_model",
        metrics: list=["mae"]
    ) -> float:
        """
        Used for cross validation on different time splits; 
        also in charge of logging every experiment run / study trial into the backend.
        """
        
        first_dates_month = pd.to_datetime(X[['year', 'month']].assign(day=1))
        train_index = first_dates_month.isin(year_month_train)
        test_index = first_dates_month.isin(year_month_test)

        X_train = X[train_index]
        X_test = X[test_index]
        y_train = y[train_index]
        y_test = y[test_index]

        # fit model on training data
        model = self.fit_model(
            X_train, 
            y_train, 
            params
        )
        # generate predictions
        y_test_pred = model.predict(X_test)
        # self.signature = infer_signature(X_train, y_test_pred)
        mae = mean_absolute_error(y_test, y_test_pred)

        mlflow.xgboost.log_model(
            model, 
            artifact_path=artifact_path,
            # signature=self.signature
        )
        mlflow.log_params(params)

        return mae

    def train_model(
        self, 
        train_df: pd.DataFrame, 
        target_col: str,
        model_name: str,
        exclude_cols: list=[],
        categorical_features: list=[],
        experiment_name: str="xgboost",
        artifact_path: str="xgboost_model",
        params: Optional[Dict]=None,
        metrics: list=["MAE"]
    ) -> None:
        """ 
        Takes an instance of `XGBRegressor` model and tracks the hyperparameter tuning
        experiment on training set using `mlflow` and `optuna`.  
        Registers the best version of the model according to a specified metric (to be implemented).
        
        -------     
        params:
        -------
        `experiment_name`: `str`
            the name of the experiment used to store runs in mlflow, 
            as well as the name of the optuna study
        `model_name`: `str`
            the name the final model will have in the registry
        `train_df`: `pd.DataFrame`
            the training data for the model.
        `target_col`: `str`
            the time-series target column
        `exclude_cols`: `list`  
            columns in dataset that should not be used
        `categorical_features`: `list` 
            list fo categorical features
        `artifact_path`: `str`
            the path pointing to the mlflow artifact
        `metrics`: `list`
            list of the metrics to track in the mlflow experiment run.
        `params`: `Optional[Dict]`
            optional dictionary of parameters to use
        """
        self.model_name = model_name

        if len(categorical_features) > 0: 
            for col in cat_columns:
                train_df[col] = df_train[col].astype('category')
        
        X = train_df.drop([target_col] + exclude_cols, axis=1)
        y = train_df[target_col]
        # unique year-month combinations -> to be used in cross-validation
        timesteps = np.sort(np.array(
            pd.to_datetime(X[['year', 'month']].assign(day=1)).unique().tolist()
        ))

        # define mlflow callback Handler for optuna 
        mlflc = MLflowCallback(
            metric_name="MAE",
        )
    
        @mlflc.track_in_mlflow() # decorator to allow mlflow logging
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 200, log=True),
                'eta': trial.suggest_float('eta', 0.01, 0.95,log=True),
                'max_depth': trial.suggest_int('max_depth', 1, 10, log=True),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 25, log=True),
                'colsample_bytree': trial.suggest_float("colsample_bytree", 0.1, 1, log=True),
                'colsample_bylevel': trial.suggest_float("colsample_bylevel", 0.1, 1, log=True),
                'colsample_bynode': trial.suggest_float("colsample_bynode", 0.1, 1, log=True),
                'subsample': trial.suggest_float("subsample", 0.5, 1, log=True),
                'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
                'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True)
            }
            cv = TimeSeriesSplit(n_splits=3) # cross validation
            cv_mae = [None]*3
            for i, (train_index, test_index) in enumerate(cv.split(timesteps)):
                cv_mae[i] = self.fit_and_test_fold(
                    params,
                    X, 
                    y, 
                    timesteps[train_index], 
                    timesteps[test_index],
                    categorical_features
                )
            trial.set_user_attr('split_mae', cv_mae)
            return np.mean(cv_mae)

        
        sampler = optuna.samplers.TPESampler(
            n_startup_trials=10, 
            seed=42
        )

        self.study = optuna.create_study(
            directions=['minimize'],
            sampler=sampler,
            study_name=experiment_name
        )

        self.study.optimize(objective, n_trials=10, timeout= 7200, callbacks=[mlflc]) 
        
        # # search for the best run at the end of the experiment # not implemented now bc of callback bug
        # best_run = mlflow.search_runs(max_results=1,order_by=["metrics.MAE"]).run_id
        # # register new model version in mlflow
        # self.result = mlflow.register_model(
        #     model_uri=f"runs:/{best_run}/{artifact_path}",
        #     name=self.model_name
        # )

    def forecast(
        self, 
        input_data: pd.DataFrame,
        use_best_from_run: bool=True,
        use_env_model: Literal["Staging", "Production", None]=None,
        use_version: int=None
        ) -> pd.DataFrame:
        """ 
        Fetches a version of the model from the mlflow backend and uses it
        to perform prediction on new input data.  
        What version is used depends on params settings, 
        defaults to using the best version from the last experiment run (currently not implemented). 
        -------     
        params:
        -------
        `input_data`: `pd.DataFrame`
            the input data for prediction,
              must have the same schema as what's in the model's signature.
        `use_best_from_run`: `bool=True`      
            use the best model from the current series of iterations, defaults to True
        `use_env_model`: `Literal["Staging", "Production", None]=None`
            use model from a given mlflow environment, defaults to None.  
            Said model might come from past iterations, depending on what you decide in the UI
        `use_version`: `int=None`
            use a previously trained version of the model. 
            Said version must have been registered from a previous iteration,  
            either by the UI or with mlflow's API
        """
        if use_best_from_run:
            # not implemented now bc of callback bug
            use_prod_model=None
            use_version=None
        
            # model = mlflow.pyfunc.load_model(
            #     model_uri=f"models:/{self.model_name}/{self.result.version}"
            # )
            # y_pred = model.predict(input_data)
            # return y_pred
        
        if use_env_model is not None:
            use_version = None

            model = mlflow.pyfunc.load_model(
                # get registered model in given environment
                model_uri=f"models:/{self.model_name}/{use_env_model}"
            )
            y_pred = model.predict(input_data)
            return y_pred

        if use_version is not None:
            # get specific registered version of model
            model = mlflow.pyfunc.load_model(
                model_uri=f"models:/{self.model_name}/{use_version}"
            )
            y_pred = model.predict(input_data)
            return y_pred

        
        if (not use_best_from_run) & (use_env_model is None) & (use_version is None):
            return ValueError(
                    "You must specify which kind of XGBoostForecaster you intend to use for prediction"
                    )

In [15]:
df_train = load_enefit_training_data()

not_feature_columns = ['datetime', 'row_id','prediction_unit_id','date','time', 'data_block_id']
cat_columns = ['county', 'product_type']

In [22]:
xgbf = XGBoostForecaster()

xgbf.train_model(
    train_df=df_train,
    target_col="target",
    model_name="xgboost_enefit",
    exclude_cols=not_feature_columns,
    categorical_features=cat_columns
)

[I 2023-11-19 11:17:26,367] A new study created in memory with name: xgboost
[W 2023-11-19 11:17:28,522] Trial 0 failed with parameters: {'n_estimators': 84, 'eta': 0.7590145927293601, 'max_depth': 5, 'min_child_weight': 5, 'colsample_bytree': 0.1432249371823025, 'colsample_bylevel': 0.14321698289111517, 'colsample_bynode': 0.1143098387631322, 'subsample': 0.9114125527116832, 'lambda': 0.25378155082656645, 'alpha': 0.6796578090758157} because of the following error: XGBoostError('[11:17:28] /Users/runner/miniforge3/conda-bld/xgboost-split_1697107917112/work/src/tree/tree_model.cc:869: Check failed: !HasCategoricalSplit(): Please use JSON/UBJSON for saving models with categorical splits.\nStack trace:\n  [bt] (0) 1   libxgboost.dylib                    0x000000013be706c0 dmlc::LogMessageFatal::~LogMessageFatal() + 140\n  [bt] (1) 2   libxgboost.dylib                    0x000000013c0372a8 xgboost::RegTree::Save(dmlc::Stream*) const + 1120\n  [bt] (2) 3   libxgboost.dylib                 

XGBoostError: [11:17:28] /Users/runner/miniforge3/conda-bld/xgboost-split_1697107917112/work/src/tree/tree_model.cc:869: Check failed: !HasCategoricalSplit(): Please use JSON/UBJSON for saving models with categorical splits.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000013be706c0 dmlc::LogMessageFatal::~LogMessageFatal() + 140
  [bt] (1) 2   libxgboost.dylib                    0x000000013c0372a8 xgboost::RegTree::Save(dmlc::Stream*) const + 1120
  [bt] (2) 3   libxgboost.dylib                    0x000000013bf79200 xgboost::gbm::GBTreeModel::Save(dmlc::Stream*) const + 304
  [bt] (3) 4   libxgboost.dylib                    0x000000013bf843f4 xgboost::LearnerIO::SaveModel(dmlc::Stream*) const + 1224
  [bt] (4) 5   libxgboost.dylib                    0x000000013be8fabc XGBoosterSaveModel + 940
  [bt] (5) 6   libffi.8.dylib                      0x0000000102df804c ffi_call_SYSV + 76
  [bt] (6) 7   libffi.8.dylib                      0x0000000102df5834 ffi_call_int

## Method 2: OneHotEncoding

In [23]:
df_train = load_enefit_training_data()

not_feature_columns = ['datetime', 'row_id','prediction_unit_id','date','time', 'data_block_id']
cat_columns = ['county', 'product_type']

In [25]:
df_encoded = pd.get_dummies(df_train, columns=cat_columns)

In [26]:
df_encoded.columns

Index(['is_business', 'target', 'is_consumption', 'datetime', 'data_block_id',
       'row_id', 'prediction_unit_id', 'date', 'time', 'year',
       'datediff_in_days', 'hour', 'hour_sine', 'hour_cosine', 'dayofweek',
       'dayofweek_sine', 'dayofweek_cosine', 'week', 'week_sine',
       'week_cosine', 'month', 'month_sine', 'month_cosine',
       'target_2_days_ago', 'eic_count', 'installed_capacity', 'euros_per_mwh',
       'lowest_price_per_mwh', 'highest_price_per_mwh', 'county_0', 'county_1',
       'county_2', 'county_3', 'county_4', 'county_5', 'county_6', 'county_7',
       'county_8', 'county_9', 'county_10', 'county_11', 'county_12',
       'county_13', 'county_14', 'county_15', 'product_type_0',
       'product_type_1', 'product_type_2', 'product_type_3'],
      dtype='object')

In [28]:
class XGBoostForecaster(Forecaster):
    """
        Implementation of a Forecaster using `XGBRegressor` as base model, 
        `optuna` for hyperparameters optimization and `mlflow` as backend to track experiments
        and register best-in-class model for time series prediction.
    """
    def __init__(self)-> None:
        self.tracking_uri = mlflow.set_tracking_uri(TRACKING_URI)
        pass

    def fit_model(
        self,  
        X:pd.DataFrame,
        y:pd.Series,
        params:Optional[Dict]=None,
    ) -> XGBRegressor:
        """
        Trains a `XGBRegressor`

        -------     
        params:
        -------
        `X`:`pd.DataFrame`
            Features to use for fitting
        `y`:`pd.Series`
            Target variable
        `params`: `Optional[Dict]`
            optional dictionary of parameters to use
        -------     
        returns:
        -------
        fitted `XGBRegressor`
        """
        model = XGBRegressor(
            n_estimators=100, 
            objective='reg:squarederror'
        )
        if params:
            model.set_params(**params)

        model.fit(X, y)
    
        return model
    
    def fit_and_test_fold(
        self, 
        params:Dict,
        X: pd.DataFrame, 
        y: pd.Series, 
        year_month_train, 
        year_month_test,
        experiment_name: str="xgboost",
        artifact_path: str="xgboost_model",
        metrics: list=["mae"]
    ) -> float:
        """
        Used for cross validation on different time splits; 
        also in charge of logging every experiment run / study trial into the backend.
        """
        
        first_dates_month = pd.to_datetime(X[['year', 'month']].assign(day=1))
        train_index = first_dates_month.isin(year_month_train)
        test_index = first_dates_month.isin(year_month_test)

        X_train = X[train_index]
        X_test = X[test_index]
        y_train = y[train_index]
        y_test = y[test_index]

        # fit model on training data
        model = self.fit_model(
            X_train, 
            y_train, 
            params
        )
        # generate predictions
        y_test_pred = model.predict(X_test)
        self.signature = infer_signature(X_train, y_test_pred)
        mae = mean_absolute_error(y_test, y_test_pred)

        mlflow.xgboost.log_model(
            model, 
            artifact_path=artifact_path,
            signature=self.signature
        )
        mlflow.log_params(params)

        return mae

    def train_model(
        self, 
        train_df: pd.DataFrame, 
        target_col: str,
        model_name: str,
        exclude_cols: list=[],
        categorical_features: list=[],
        experiment_name: str="xgboost",
        artifact_path: str="xgboost_model",
        params: Optional[Dict]=None,
        metrics: list=["MAE"]
    ) -> None:
        """ 
        Takes an instance of `XGBRegressor` model and tracks the hyperparameter tuning
        experiment on training set using `mlflow` and `optuna`.  
        Registers the best version of the model according to a specified metric (to be implemented).
        
        -------     
        params:
        -------
        `experiment_name`: `str`
            the name of the experiment used to store runs in mlflow, 
            as well as the name of the optuna study
        `model_name`: `str`
            the name the final model will have in the registry
        `train_df`: `pd.DataFrame`
            the training data for the model.
        `target_col`: `str`
            the time-series target column
        `exclude_cols`: `list`  
            columns in dataset that should not be used
        `artifact_path`: `str`
            the path pointing to the mlflow artifact
        `metrics`: `list`
            list of the metrics to track in the mlflow experiment run.
        `params`: `Optional[Dict]`
            optional dictionary of parameters to use
        """
        self.model_name = model_name

        if len(categorical_features) > 0: 
           train_df = pd.get_dummies(train_df, columns=cat_columns)

        X = train_df.drop([target_col] + exclude_cols, axis=1)
        y = train_df[target_col]
        # unique year-month combinations -> to be used in cross-validation
        timesteps = np.sort(np.array(
            pd.to_datetime(X[['year', 'month']].assign(day=1)).unique().tolist()
        ))

        # define mlflow callback Handler for optuna 
        mlflc = MLflowCallback(
            metric_name="MAE",
        )
    
        @mlflc.track_in_mlflow() # decorator to allow mlflow logging
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 200, log=True),
                'eta': trial.suggest_float('eta', 0.01, 0.95,log=True),
                'max_depth': trial.suggest_int('max_depth', 1, 10, log=True),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 25, log=True),
                'colsample_bytree': trial.suggest_float("colsample_bytree", 0.1, 1, log=True),
                'colsample_bylevel': trial.suggest_float("colsample_bylevel", 0.1, 1, log=True),
                'colsample_bynode': trial.suggest_float("colsample_bynode", 0.1, 1, log=True),
                'subsample': trial.suggest_float("subsample", 0.5, 1, log=True),
                'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
                'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True)
            }
            cv = TimeSeriesSplit(n_splits=3) # cross validation
            cv_mae = [None]*3
            for i, (train_index, test_index) in enumerate(cv.split(timesteps)):
                cv_mae[i] = self.fit_and_test_fold(
                    params,
                    X, 
                    y, 
                    timesteps[train_index], 
                    timesteps[test_index]
                )
            trial.set_user_attr('split_mae', cv_mae)
            return np.mean(cv_mae)

        
        sampler = optuna.samplers.TPESampler(
            n_startup_trials=10, 
            seed=42
        )

        self.study = optuna.create_study(
            directions=['minimize'],
            sampler=sampler,
            study_name=experiment_name
        )

        self.study.optimize(objective, n_trials=50, timeout= 7200, callbacks=[mlflc]) 
        
        # # search for the best run at the end of the experiment # not implemented now bc of callback bug
        # best_run = mlflow.search_runs(max_results=1,order_by=["metrics.MAE"]).run_id
        # # register new model version in mlflow
        # self.result = mlflow.register_model(
        #     model_uri=f"runs:/{best_run}/{artifact_path}",
        #     name=self.model_name
        # )

    def forecast(
        self, 
        input_data: pd.DataFrame,
        use_best_from_run: bool=True,
        use_env_model: Literal["Staging", "Production", None]=None,
        use_version: int=None
        ) -> pd.DataFrame:
        """ 
        Fetches a version of the model from the mlflow backend and uses it
        to perform prediction on new input data.  
        What version is used depends on params settings, 
        defaults to using the best version from the last experiment run (currently not implemented). 
        -------     
        params:
        -------
        `input_data`: `pd.DataFrame`
            the input data for prediction,
              must have the same schema as what's in the model's signature.
        `use_best_from_run`: `bool=True`      
            use the best model from the current series of iterations, defaults to True
        `use_env_model`: `Literal["Staging", "Production", None]=None`
            use model from a given mlflow environment, defaults to None.  
            Said model might come from past iterations, depending on what you decide in the UI
        `use_version`: `int=None`
            use a previously trained version of the model. 
            Said version must have been registered from a previous iteration,  
            either by the UI or with mlflow's API
        """
        if use_best_from_run:
            # not implemented now bc of callback bug
            use_prod_model=None
            use_version=None
        
            # model = mlflow.pyfunc.load_model(
            #     model_uri=f"models:/{self.model_name}/{self.result.version}"
            # )
            # y_pred = model.predict(input_data)
            # return y_pred
        
        if use_env_model is not None:
            use_version = None

            model = mlflow.pyfunc.load_model(
                # get registered model in given environment
                model_uri=f"models:/{self.model_name}/{use_env_model}"
            )
            y_pred = model.predict(input_data)
            return y_pred

        if use_version is not None:
            # get specific registered version of model
            model = mlflow.pyfunc.load_model(
                model_uri=f"models:/{self.model_name}/{use_version}"
            )
            y_pred = model.predict(input_data)
            return y_pred

        
        if (not use_best_from_run) & (use_env_model is None) & (use_version is None):
            return ValueError(
                    "You must specify which kind of XGBoostForecaster you intend to use for prediction"
                    )
        

In [29]:
df_train = load_enefit_training_data()

not_feature_columns = ['datetime', 'row_id','prediction_unit_id','date','time', 'data_block_id']
cat_columns = ['county', 'product_type']

In [30]:
xgbf = XGBoostForecaster()

xgbf.train_model(
    train_df=df_train,
    target_col="target",
    model_name="xgboost_enefit",
    exclude_cols=not_feature_columns,
    categorical_features=cat_columns
)

[I 2023-11-19 11:42:00,222] A new study created in memory with name: xgboost
[I 2023-11-19 11:42:30,915] Trial 0 finished with value: 176.29408523071473 and parameters: {'n_estimators': 84, 'eta': 0.7590145927293601, 'max_depth': 5, 'min_child_weight': 5, 'colsample_bytree': 0.1432249371823025, 'colsample_bylevel': 0.14321698289111517, 'colsample_bynode': 0.1143098387631322, 'subsample': 0.9114125527116832, 'lambda': 0.25378155082656645, 'alpha': 0.6796578090758157}. Best is trial 0 with value: 176.29408523071473.
[I 2023-11-19 11:42:55,161] Trial 1 finished with value: 206.06682436048575 and parameters: {'n_estimators': 51, 'eta': 0.8283494908181351, 'max_depth': 6, 'min_child_weight': 1, 'colsample_bytree': 0.1519934830130981, 'colsample_bylevel': 0.15254729458052607, 'colsample_bynode': 0.20148477884158655, 'subsample': 0.7193453335958095, 'lambda': 0.05342937261279776, 'alpha': 0.014618962793704969}. Best is trial 0 with value: 176.29408523071473.
[I 2023-11-19 11:43:19,824] Trial 